In [14]:
# import the libraries as shown below

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [15]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\train'
valid_path = r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\test'

In [16]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [17]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [18]:
# useful for getting number of output classes
folders = glob(r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\train\*')

In [19]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [20]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [21]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [22]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [23]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [24]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [25]:
test_set = test_datagen.flow_from_directory(r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [26]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

163/163 [==============================] - 1797s 11s/step - loss: 0.2277 - accuracy: 0.9072 - val_loss: 0.4453 - val_accuracy: 0.8846


In [27]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('Pneumonia_model_vgg16.h5')

In [8]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
model0=load_model('Pneumonia_model_vgg16.h5')
#img=image.load_img(r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\train\NORMAL\IM-0115-0001.jpeg',target_size=(224,224))
img=image.load_img(r'D:\deep learning\Chest X-Ray Images (Pneumonia)\chest_xray\chest_xray\test\PNEUMONIA\person1_virus_6.jpeg',target_size=(224,224))

x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
classes=model0.predict(img_data)
print(classes)
if(np.argmax(classes)==0):
    print("Normal.")
else:
    print("Pneumonia")

[[0. 1.]]
Pneumonia
